# How to Create a US Census Bar Chart Race
## Overview
The beginning of every data science project starts with data and this will be the focus of this project. Usually, this part involves three steps:
 * data acquisition,
 * data cleaning and
 * data visualization.
 
Data visualization can mean anything from simple print statements to fancy plots and animations.

## Data Acquisition
As a data source, we use Wikipedia's census data websites. Since this data is only available on the web, we need to parse the website and extract the useful data. We use the popular [pandas](https://pandas.pydata.org/) package for this task. 

In [13]:
import pandas as pd

We also be using two more packages, `pickle` which allows us to easily save data to disk and `re` which allows us to use [regular expression](https://regexr.com/) syntax. Finally, for nicer table formatting, we import `IPython.display`.

In [14]:
import pickle
import re
from IPython.display import display

# Define empty Census data dictionary - the keys will be the census years.
Census_raw = {}

### Parsing Wikipedia
Generally, parsing is not recommended to get data from the web as we often have to deal with messy formatting and a lot of cleaning up is needed. However, in this case, Wikipedia's layout of the US census years seems relatively consistent and thus we give parsing a shot. The following code goes through all Wikipedia census websites from [1790](https://en.wikipedia.org/wiki/1790_United_States_Census) to [2010](https://en.wikipedia.org/wiki/2010_United_States_Census) and collects all tables from the pages. We also add the latest census estimate from 2019.

In [15]:
# Uncomment if you like to parse the data again (not recommended!)

# for year in range(1790,2020,10):
#     Census_raw[year]=pd.read_html('https://en.wikipedia.org/wiki/{}_United_States_Census'.format(year))
# Census_raw[2019]=pd.read_html('https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States_by_population')

Once, we have parsed the data for the first time, we save it to disk and when we run our notebook the next time, we can skip the parsing step.

In [16]:
if len(Census_raw.keys()):
    with open('rawdata.pickle', 'wb') as handle:
        pickle.dump(Census_raw, handle, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open('rawdata.pickle', 'rb') as handle:
        Census_raw = pickle.load(handle)

## Data Cleaning

All Census pages contain several tables but we are only interested in those tables that contain `State` (or `District` as in [1800](https://en.wikipedia.org/wiki/1800_United_States_Census)) as on of their columns. Most pages also have a seperate statistic on the largest cities and we thus ignore all tables that have `City` as on of their columns.

In [17]:
Census = {}
for key in Census_raw.keys():
    Census[key] = [table for table in Census_raw[key] if ('State' in table.columns) or (('District' in table.columns))]
    if len(Census[key])>1:
        Census[key] = [table for table in Census[key] if ('City' not in table.columns)][0]
    else:
        Census[key] = Census[key][0]

After these relatively easy manipulations, we need to get our hands dirty and make sure that all tables are of the correct format. Often, it is best to visaualize the tables (or at least the first entry) to get a better idea. So let's do that.After these relatively easy manipulations, we need to get our hands dirty and make sure that all tables are of the correct format. Often, it is best to visualize the tables (or at least the first entry) to get a better idea. So let's do that. 

In [18]:
for key in Census.keys():
    print("Displaying values for {}".format(key))
    display(Census[key].head(1))

Displaying values for 1790


,State,"Free white males of 16 years and upward, including heads of families",Free white males under 16 years,"Free white females, including heads of families",All other free persons,Slaves,Enslaved % of state population,Total,% of US population
0,Vermont,22435,22328,40505,255,16[a][7],0.02%,"85,539[b]",2.2%


Displaying values for 1800


,District,"Free white males under age 10,",Free white males age 10–16,Free white males age 16–26,Free white males age 26–45,Free white males over age 45,Free white females under age 10,Free white females age 10–16,Free white females age 16–26,Free white females age 26–45,Free white females over age 45,All other free persons,Slaves,Total
0,New Hampshire,30694,14881,16379,17589,11715,29871,14193,17153,18381,12142,852,8,183858


Displaying values for 1810


...

,Rank,State,Population as of1990 Census[4],Population as of2000 Census[4],Change,Percentchange
0,1,California,29760021,33871648,4111627,13.8%


Displaying values for 2010


,Rank,State,Population as of2000 Census,Population as of2010 Census[51],Change,Percentchange
0,1,California,33871648,37253956,3382308,10.0%


Displaying values for 2019


,"Rank in the fifty states, 2019","Rank in states & territories, 2010",State,"Population estimate, July 1, 2019[5]","Census population, April 1, 2010[6]","Percent change, 2010–2019[note 1]","Absolute change, 2010-2019","Total seats in the U.S. House of Representatives, 2013–2023","Estimated population per electoral vote, 2019[note 2]","Estimated population per House seat, 2019","Census population per House seat, 2010","Percent of the total U.S. population, 2018[note 3]"
0,1.0,1.0,California,39512223,37254523,6.1%,"+2,257,700",53,718404,745514,702885,11.96%


Ok, so it looks like 1790 and 1800 use `Total` instead of `Population` and 1800 also uses `District` instead of `State`. Let's change that! 

In [19]:
Census[1790].rename(columns={'Total':'Population'},inplace=True)
Census[1800].rename(columns={'Total':'Population', 'District':'State'},inplace=True)

Also, from 1990 to 2010, the population of the previous decade is listed and we need to make sure we get the right one. For this task, we use regular expressions to select the column that contains the current year. Once we have found the right column, we rename it to `Population`. For 2019, the need to change it from `Population estimate, July 1, 2019[5]` to `Population`.

In [20]:
for year in range(1990,2020,10):
    d = {'Population as of{}.*'.format(year): 'Population'}
    Census[year].columns = Census[year].columns.to_series().replace(d, regex=True)
Census[2019].columns = Census[2019].columns.to_series().replace({'Population estimate, July 1, 2019.*': 'Population'}, regex=True)

Next, we get rid of all columns other than `State` and `Population`. We also remove all non-numerical characters from the `Population` column with some regular expression magic. Once we have down these two steps, we display the first 10 entries of all tables.

In [21]:
for key in Census.keys():
    print("Displaying values for {}".format(key))
    Census[key] = Census[key][['State','Population']]
    Census[key].loc[:,'Population']=Census[key].loc[:,'Population'].apply(
        lambda s: int(re.sub(r'\[.*','', str(s)).replace(',','')))
    display(Census[key].head(10))

Displaying values for 1790


,State,Population
0,Vermont,85539
1,New Hampshire,141885
2,Maine,96540
3,Massachusetts,378787
4,Rhode Island,68825
5,Connecticut,237946
6,New York,340120
7,New Jersey,184139
8,Pennsylvania,434373
9,Delaware,59094


Displaying values for 1800


,State,Population
0,New Hampshire,183858
1,Massachusetts,422845
2,Maine,151719
3,Connecticut,251002
4,Vermont,154465
5,Rhode Island,69122
6,"New York (excluding Duchess, Ulster, Orange co...",484065
7,"New York (Duchess, Ulster, Orange counties)",101985
8,New Jersey,211149
9,Pennsylvania (eastern district),327979


Displaying values for 1810


...

,State,Population
0,California,33871648
1,Texas,20851820
2,New York,18976457
3,Florida,15982378
4,Illinois,12419293
5,Pennsylvania,12281054
6,Ohio,11353140
7,Michigan,9938444
8,New Jersey,8414350
9,Georgia,8186453


Displaying values for 2010


,State,Population
0,California,37253956
1,Texas,25145561
2,New York,19378102
3,Florida,18801310
4,Illinois,12830632
5,Pennsylvania,12702379
6,Ohio,11536504
7,Michigan,9883640
8,Georgia,9687653
9,North Carolina,9535483


Displaying values for 2019


,State,Population
0,California,39512223
1,Texas,28995881
2,Florida,21477737
3,New York,19453561
4,Pennsylvania,12801989
5,Illinois,12671821
6,Ohio,11689100
7,Georgia,10617423
8,North Carolina,10488084
9,Michigan,9986857


## The year 1800 😒

In [22]:
display(Census[1800].head(10))

,State,Population
0,New Hampshire,183858
1,Massachusetts,422845
2,Maine,151719
3,Connecticut,251002
4,Vermont,154465
5,Rhode Island,69122
6,"New York (excluding Duchess, Ulster, Orange co...",484065
7,"New York (Duchess, Ulster, Orange counties)",101985
8,New Jersey,211149
9,Pennsylvania (eastern district),327979


Unfortunately, the year 1800 doesn't play well with the other years and requires some additional cleaning up (that's why parsing is not recommended). We can see that instead of States, the 1800 census data has smaller subdivisions. To remedy this, we match our list of the 50+1 states against the district names and rename them to a single state. For example, our row with `New York (Duchess, Ulster, Orange counties)` will become just `New York`. Once we have renamed them, we group them by state and sum up the populations on all rows that share the same state. Just to be safe, we apply the same procedure to every census year. Grouping also has the nice side effect of setting the index to our column that we group by (`State`). Finally, we rename our `Population` column to the census year. 

In [23]:
States = pd.read_csv('States.csv')['States']
StatesList = States.tolist()

if isinstance(Census[1790].index[0],int):
    for key in Census.keys():
        for i, row in Census[key].iterrows():
            for State in StatesList:
                if State in row['State']:
                    Census[key].loc[i,'State'] = State  
        Census[key] = Census[key][Census[key]['State'].isin(StatesList)]
        Census[key] = Census[key].groupby(['State'],squeeze=True).sum()
        Census[key].rename(columns={'Population':str(key)},inplace=True)
display(Census[1800].head(10))

,1800
State,
Connecticut,251002
Delaware,64273
District of Columbia,323297
Georgia,162686
Indiana,5641
Kentucky,220959
Maine,151719
Maryland,4276
Massachusetts,422845


It worked! Great, now we are ready to put everything in one big table and then use it to visualize the census data over the past 230 years!

In [24]:
result_df = pd.DataFrame(States).set_index('States')
for key in Census.keys():
    result_df = result_df.add(Census[key],fill_value=0)
result_df.to_csv('State_Census_Historical.csv')
result_df.head(5)

,1790,1800,1810,1820,1830,1840,1850,1860,1870,1880,...,1930,1940,1950,1960,1970,1980,1990,2000,2010,2019
Alabama,NaN,NaN,9046.0,127901.0,309527.0,590756.0,771623.0,964201.0,996992.0,1315497.0,...,2646248.0,2832961.0,3060000.0,3273000.0,3444165.0,3893888.0,4040587.0,4447100.0,4779736.0,4903185.0
Alaska,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,59278.0,72524.0,138000.0,200000.0,302173.0,401851.0,550043.0,626932.0,710231.0,731545.0
Arizona,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9658.0,62266.0,...,435573.0,499261.0,756000.0,1318000.0,1772482.0,2718215.0,3665228.0,5130632.0,6392017.0,7278717.0
Arkansas,NaN,NaN,1062.0,14273.0,30388.0,97574.0,209897.0,435450.0,484471.0,802525.0,...,1854482.0,1949387.0,1906000.0,1788000.0,1923295.0,2286435.0,2350725.0,2673400.0,2915918.0,3017825.0
California,NaN,NaN,NaN,NaN,NaN,NaN,92597.0,379994.0,560247.0,864694.0,...,5677251.0,6907387.0,10586223.0,15850000.0,19953134.0,23667902.0,29760021.0,33871648.0,37253956.0,39512223.0


## Data Visualization
Getting and cleaning the data was by far the hardest part. Now that we got this out of the way, we can use a visualization tool such as Flourish to get our final [**US Census Bar Chart Race**](https://public.flourish.studio/visualisation/1322083/) which is available at https://public.flourish.studio/visualisation/1322083.

Congrats on completing this tutorial and I hope it can help you in your next data science project!

## Licensing 
This work is licensed under a [Creative Commons Attribution 4.0 International
License][cc-by]. This means that you are free to:

 * **Share** — copy and redistribute the material in any medium or format
 * **Adapt** — remix, transform, and build upon the material for any purpose, even commercially.

Under the following terms:

 * **Attribution** — a link to this [github repo](https://github.com/AntonMu/Census2020)!
 
[![CC BY 4.0][cc-by-image]][cc-by]

[cc-by]: http://creativecommons.org/licenses/by/4.0/
[cc-by-image]: https://i.creativecommons.org/l/by/4.0/88x31.png
[cc-by-shield]: https://img.shields.io/badge/License-CC%20BY%204.0-lightgrey.svg